<a href="https://colab.research.google.com/github/asigalov61/Meddleying-VIRTUOSO/blob/main/Meddleying_VIRTUOSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Meddleying VIRTUOSO (ver 1.0)

***

## Full-featured Algorithmic Intelligence Music Augmentator (AIMA)

PLEASE NOTE: IT ONLY WORKS FOR PIANO ONLY FOR NOW !!!

***

### Project Los Angeles

### Tegridy Code 2020

***

# Setup Environment, clone needed code, and install all required dependencies

In [ ]:
#@title Install all dependencies (run only once per session)
print('Installing dependencies and creating Dataset dir...')
!pip install pretty_midi
!pip install visual_midi

!curl -L "https://raw.githubusercontent.com/asigalov61/Meddleying-MAESTRO/main/MIDI.py" > 'MIDI.py'

!mkdir '/content/Dataset/'
print('Done!')


In [ ]:
#@title Import all modules
print('Importing modules...')
import glob
import os
import numpy as np
import toolz
import itertools
import music21
from music21 import *
import pickle
import time
import math
import sys
import tqdm.auto
import secrets
import pretty_midi

import statistics
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
from mido import MidiFile
from IPython.display import display, Image
import MIDI
from visual_midi import Plotter
from visual_midi import Preset
from pretty_midi import PrettyMIDI

debug = False
print('Done!')

# Select and download a sample MIDI dataset

In [ ]:
#@title (BEST Choice / Multi-Intrumental) Special Tegridy MIDI Multi-Instrumental Dataset (1967 MIDIs)
%cd /content/Dataset/
!wget 'https://github.com/asigalov61/Meddleying-MAESTRO/raw/main/Dataset/Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!unzip -j 'Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
!rm 'Tegridy-MIDI-Dataset-CC-BY-NC-SA.zip'
%cd /content/

In [ ]:
#@title (Piano Performance Dataset) Download Google Magenta MAESTRO v.2.0.0 Piano MIDI Dataset (~1300 MIDIs)
%cd /content/Dataset/
!wget 'https://storage.googleapis.com/magentadata/datasets/maestro/v2.0.0/maestro-v2.0.0-midi.zip'
!unzip -j maestro-v2.0.0-midi.zip
!rm maestro-v2.0.0-midi.zip
%cd /content/

In [ ]:
#@title A simple code to unzip MIDI datasets without any hastle
!mkdir /content/Dataset/
%cd /content/Dataset/
!unzip -j /content/Dataset/*.zip
!rm /content/Dataset/*.zip
%cd /content/

# Process MIDI Dataset to MIDI Notes and MIDI Events Lists

In [ ]:
#@title Do not forget to add your MIDIs you will be augmenting to the dataset!
print('Meddleying VIRTUOSO Algorithmic Intelligence Music Augmentator')
print('Starting up...')
full_path_to_output_dataset_to = "/content/Meddleying-VIRTUOSO-Music-Dataset.data" #@param {type:"string"}
show_detailed_dataset_stats_and_3D_plot = False #@param {type:"boolean"}


##debug = False #@param {type:"boolean"}

###########

os.chdir("./")

###########


average_note_pitch = 0
min_note = 127
max_note = 0

files_count = 0
melody_list = []
chords_events = []
chords_list = []

pitches_list = []
durations_list = []
events_matrix = []
file_matrix = []
chords_list = []
ev = 0

###########

def list_average(num):
  sum_num = 0
  for t in num:
      sum_num = sum_num + t           

  avg = sum_num / len(num)
  return avg

###########

print('Loading MIDI files...')
print('This may take a while on a large dataset in particular.')


dataset_addr = "Dataset"

files = os.listdir(dataset_addr)
print('Now processing the files.')
print('Please stand-by...')

for file in tqdm.auto.tqdm(files):
    file_address = os.path.join(dataset_addr, file)
    
    score = []

    midi_file = open(file_address, 'rb')
    if debug: print('Processing File:', file_address)
    try:
      opus = MIDI.midi2opus(midi_file.read())
    except:
      print('Bad file. Skipping...')
      midi_file.close()
      continue
    midi_file.close()

    score1 = MIDI.to_millisecs(opus)
    score2 = MIDI.opus2score(score1)
    score = MIDI.grep(score2, [0])

    zero_event = ['note', 0, 0, 0, 0, 0] # Zero note MIDI.py event
    previous_event = ['note', 0, 0, 0, 0, 0] # Zero note previous MIDI.py event
    time = 0
    itrack = 1
    while itrack < len(score):
      chords_events.append(zero_event)
      for event in score[itrack]:
        if event[0] == 'note':
          event[3] = 0
          file_matrix.append([event, event[1]])
          events_matrix.append(event)
          pitches_list.append(event[4])
          durations_list.append(event[2])
          min_note = int(min(min_note, event[4]))
          max_note = int(max(max_note, event[4]))          

         
      itrack += 1
      time = 0  
    values = set(map(lambda x:x[1], file_matrix))
    chords_list.extend([[y[0] for y in file_matrix if y[1]==x] for x in values])
  
    file_matrix = []
    files_count += 1

print('Indexing the resulting dataset...')
y  = []
for i in range(len(chords_list)):
  if len(chords_list[i]) < 2:
    y.append(i)

print('Sorting the dataset...')
y.sort(reverse=True)

print('Removing single notes from the dataset... ')
for z in y:
  chords_list.pop(z)

print('Final sorting...')
chords_list.sort(key=lambda x: x[0][1])

print('Doing some math...')
average_note_pitch = int((min_note + max_note) / 2)

print('Task complete :)')
print('==================================================')
print('Number of processed dataset MIDI files:', files_count)
print('Average note pitch =', average_note_pitch)
print('Min note pitch =', min_note)
print('Max note pitch =', max_note)
print('Number of MIDI events recorded:', len(pitches_list))
print('Number of MIDI chords recorded:', len(chords_list))
# Dataset
MusicDataset = [chords_list]
print('Writing dataset to disk...')
with open(full_path_to_output_dataset_to, 'wb') as filehandle:
    # store the data as binary data stream
    pickle.dump(MusicDataset, filehandle)
print('Dataset was saved at:', full_path_to_output_dataset_to)
if show_detailed_dataset_stats_and_3D_plot:

  print('Detailed dataset stats:')

  pd_stats_matrix = [ [0 for col in range(128)] for col in range(128)]

  for event in events_matrix:
    if event[4] < 128 and int(event[2] / 128) < 128:
      pd_stats_matrix[event[4]][int(event[2] / 128)] += 1

  data = np.array(pd_stats_matrix)
  length = data.shape[0]
  width = data.shape[1]
  x, y = np.meshgrid(np.arange(length), np.arange(width))

  fig = plt.figure()
  fig.dpi = 200
  ax = fig.add_subplot(1,1,1, projection='3d')
  ax.plot_surface(x, y, data)
  ax.view_init(60, 60)
  #plt.show()
  plt.savefig('Meddleying-VIRTUOSO-Dataset-3D-graph.jpg', dpi=600)

  print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in pd_stats_matrix]))


print('Task complete. Enjoy! :)')

# Load/Re-load the processed dataset

In [ ]:
#@title Load/Reload pre-processed dataset from a file to memory
full_path_to_dataset_file = "/content/Meddleying-VIRTUOSO-Music-Dataset.data" #@param {type:"string"}

chords_list = []
melody_list = []
print('Loading Meddleying VIRTUOSO Dataset')
with open(full_path_to_dataset_file, 'rb') as filehandle:
    # read the data as binary data stream
    MusicDataset = pickle.load(filehandle)
    #melody_list = MusicDataset[0]
    chords_list= MusicDataset[0]
print('Task complete. Enjoy! :)')
print('==================================================')
#print('Number of melody notes in the dataset:', len(melody_list))
print('Number of chords in the dataset:', len(chords_list))
print('Done! Enjoy! :)')   

# Generate Music

## VERY IMPORTANT! YOU MUST PRIME FROM THE DATASET
### This means that you MUST include the input MIDI files you want to augment into the Dataset before generating a composition. There is no way around it atm and ALL SOTA Music AI systems employ the same technique.

In [ ]:
input_melody_list

In [ ]:
#@title Generate Music
file_address = "/content/Smooth Melody #1.mid" #@param {type:"string"}
output_ticks = 1000
trim_the_output_to_this_many_melody_notes = 300 #@param {type:"slider", min:10, max:2000, step:10}
min_notes_per_chord = 3 #@param {type:"slider", min:2, max:10, step:1}
max_notes_per_chord = 6 #@param {type:"slider", min:2, max:10, step:1}
melody_only_output = True #@param {type:"boolean"}
chords_only_output = True #@param {type:"boolean"}
chords_durations_multiplier = 2 #@param {type:"slider", min:0.1, max:3, step:0.1}
transpose_melody_velocity = 10 #@param {type:"slider", min:-30, max:30, step:1}
transpose_chords_velocity = 0 #@param {type:"slider", min:-30, max:30, step:1}
random_chords = True #@param {type:"boolean"}
random_coefficient_in_search_attempts = 2 #@param {type:"slider", min:2, max:50, step:1}
output_input_melody_only = False #@param {type:"boolean"}
output_input_chords_only = False #@param {type:"boolean"}

print('Meddleying VIRTUOSO Algorithmic Intelligence Music Agumentator.')
print('Starting up...')
output_composition = []


print('Loading input MIDI file...')
score = []
output_composition = []

midi_file = open(file_address, 'rb')
print('Processing File:', file_address)
try:
  opus = MIDI.midi2opus(midi_file.read())
except:
  print=('Bad file. Please check and try again...')
  midi_file.close()
  exit
midi_file.close()

score1 = MIDI.to_millisecs(opus)
score2 = MIDI.opus2score(score1)
score = MIDI.grep(score2, [0])
flattened_chords_list = list(itertools.chain(*list(itertools.chain(chords_list)))) 

zero_event = ['note', 0, 0, 0, 0, 0] # Zero note MIDI.py event
previous_event = ['note', 0, 0, 0, 0, 0] # Zero note previous MIDI.py event

print('Processing input MIDI file...')

melody_chords_events = []
melody_chords_list = []
melody_events = []

print('Extracting melody...')

itrack = 1   # skip 1st element which is ticks
while itrack < len(score):
    for event in tqdm.auto.tqdm(score[itrack]):
      if event[0] == 'note':

        event[3] = 0

        if event[1] >= previous_event[1]:
          melody_chords_events.append(event)              
        else:
          melody_chords_events.sort(key=lambda x: x[4])
          if melody_chords_events != []: 
            melody_events.append(melody_chords_events)            
          melody_chords_events = []
          melody_chords_events.append(event)

        previous_event = event             

    previous_event = zero_event
    melody_chords_events = []          
    itrack += 1

input_chords_list = [i for i in melody_events if i != []]
input_melody_list = [event[0] for event in melody_events]
input_chords_only_list = [i for i in [i[1:] for i in melody_events ] if i != []]
melody_events = list(itertools.chain(*list(itertools.chain(melody_events)))) 

input_melody_list.sort(key=lambda x: x[1])


print('Generating composition...')
print('Please stand by...')

i = 0
dts = 0
c = 0
h = 0
output_mchord = []
if not output_input_melody_only and not output_input_chords_only:
  for event in tqdm.auto.tqdm(input_melody_list):
    if i <= trim_the_output_to_this_many_melody_notes:
      
      if melody_only_output:
        event1 = event
        event1[5] = event[4] + transpose_melody_velocity
        event1[3] = 0
        output_composition.append([event1])
      i +=1

      for chord in chords_list:
        if len(chord) <= max_notes_per_chord and len(chord) > 1:

          if event[4] == chord[0][4]:          
            if event[2] == chord[0][2]:

                        
              for cev in chord:
                cev[1] = event[1]
                cev[3] = 0
                cev[2] = cev[2] * chords_durations_multiplier
                cev[5] = event[4] + transpose_chords_velocity  
                output_mchord.append(cev)
                h += 1

              if random_chords:
                if secrets.randbelow(random_coefficient_in_search_attempts) == 0:
                  random_chords = False
                else:
                  random_chords = True  
              
              if chords_only_output:
                if not random_chords:  
                  output_composition.append(output_mchord[1:])
                  c += 1
                  output_mchord = [] 
                else:
                  output_mchord = []         
                  break
              else:
                output_composition.append(output_mchord[1:])
                c += 1
                output_mchord = []
                break          
              output_mchord = []         
              break
            else:
              continue
          else:
            continue
        else:
          continue
          
      dts += 1                
      previous_event = event

print('Done!')
print('Processing output melody and chords lists')
if not output_input_chords_only and not output_input_melody_only:
  output_composition = [i for i in output_composition if i != [] or i[4] != 0] #or i[5] != 0]
  output = list(itertools.chain(*list(itertools.chain(output_composition[:trim_the_output_to_this_many_melody_notes]))))

if output_input_melody_only:
  output = list(itertools.chain(*list(itertools.chain([input_melody_list]))))

if output_input_chords_only:
  output = list(itertools.chain(*list(itertools.chain(input_chords_list))))

print('Generating output MIDI file...')
output_ticks = score[0]
output_header = [['track_name', 0, b'Meddleying VIRTUOSO']] + score[1][:10] + [['patch_change', 0, 0, 0]]
output_score = output
output_opus = output_header + output_score
output = [output_ticks,  output_opus]

midi_data = MIDI.score2midi(output)
print('Writing output MIDI file to disk...')
with open('Meddleying-VIRTUOSO-Composition.mid', 'wb') as midi_file1:
    midi_file1.write(midi_data)
    midi_file1.close()
print('Done!')

print('Crunching quick stats...')
print('First Event:', output[1][0], '=== Last Event:', output[1][-1])
print('The dataset was scanned', dts, 'times.')
print('Generated', i, 'notes for', len(melody_events), 'input melody notes.')
print('Generated', c, 'chords for', len(melody_events), 'input melody notes.')
print('Task complete!')

print('Downloading your MIDI now :) Enjoy!')
from google.colab import files
files.download('/content/Meddleying-VIRTUOSO-Composition.mid')
print('Enjoy! :)')

print('Final output MIDI stats:')
MIDI.score2stats(output)

# Fun MIR stats

In [ ]:
#@title Basic statistical analysis of the output MIDI file

MIDI_DIR = "/content/*.mid"
### https://github.com/brennan2602/FYP


def get_piano_roll(midifile):
	midi_pretty_format = pretty_midi.PrettyMIDI(midifile)
	piano_midi = midi_pretty_format.instruments[0] # Get the piano channels
	piano_roll = piano_midi.get_piano_roll(fs=20)
	return piano_roll

#uses split encoding scheme (here only encoding the note values)
#works by looping through time increments of the piano roll array and writing the notes being played
#at a given time sample as a number on the corresponding line of a string # is written when no notes played for that
#sample
def encode(arr):
    timeinc=0
    outString=""
    for time in arr:
        notesinc = -1
        #print(time)
        if np.all(time==0):
            outString=outString+"#"
        for vel in arr[timeinc]:
            notesinc=notesinc+1
            if vel != 0:
                noteRep=str(notesinc) + " "
                #print(noteRep)
                outString=outString+noteRep
        outString=outString+"\n"
        timeinc = timeinc+1
    return outString


def getSilences(test):
    test=test[:-1] #removing last line in string (always blank)
    output=test.split("\n") #splitting into array
    res = len(output)
    #initialising counters
    maxcounter=0
    counter=0
    silenceCount=0

    for x in output:
        if x == "#": #when a "#" is seen nothing is being played that sample
            counter=counter+1 #this tracks a streak of silences
            silenceCount+=1 #this tracks total silences
        if x != "#":
            counter=0 #reseting streak
        if counter>maxcounter:
            maxcounter=counter #updating longest silence streak when appropriate
    return maxcounter,silenceCount


#by looking at the length of song and the amount of silences this returns % silence
def getPercentSilence(gen,silences):
    test = gen
    test = test[:-1]
    output = test.split("\n")
    res = len(output)
    percent=silences/res
    return percent


def getStatsNotes(test):
    test=test[:-1] #get rid of blank line at the end
    notes=[]
    output = test.split("\n") #split string on new lines

    #initial values updated while looping through
    maxPerSamp=0
    silenceSamp=0
    notesPlayed=0
    maxNotes=0
    maxVal=0
    minVal=127

    for x in output:
        samp=x.split(" ")
        samp=samp[:-1] #theres a blank result at the end of array from split this indexing removes it
        while "0" in samp:
            samp.remove("0") #sometimes 0 samples exist this removes them as they aren't notes played
        if len(samp)==0:
            silenceSamp+=1 #counting silences
        notesPlayed=notesPlayed+len(samp) #counting notes played
        if len(samp)>0:
            #getting max and min note values at this time step
            minimum=min(samp)
            maximum=max(samp)
            #updating max and min values note values for song if appropriate
            if int(minimum)<minVal:
                minVal=int(minimum)
            if int(maximum)>maxVal:
                maxVal=int(maximum)
        #updating maximum number of notes per sample if appropriate
        if len(samp)>maxNotes:
            maxNotes=len(samp)
    rangeNotes=maxVal-minVal #spread of notes
    avgNotes = notesPlayed / len(output) #average notes per sample
    adjNotes=notesPlayed /(len(output)-silenceSamp) #average notes per sample adjusted to remove silent samples
    return rangeNotes, maxVal, minVal,maxNotes,avgNotes,adjNotes


files=glob.glob(MIDI_DIR)#point towards directory with midi files (here same folder)
#print(files)

for f in files:
    print(f)
    pr = get_piano_roll(f) #gets piano roll representation of the midi file
    arr = pr.T
    outString= encode(arr) #gets a string representation of the midi file
    maxsilences, silences = getSilences(outString) #by passing in the encoded string get longest silence and the total
                                                   #number of samples which are silent
    noteRange, maxVal, minVal, maxNotes, avgNotes, adjAvg =getStatsNotes(outString) # getting some stats by looping
                                                                                    # through encoded data
    percentSilence= getPercentSilence(outString,silences) # get % silence from silence / outString length

    #printing out to the user
    print("longest silence is ",maxsilences,"samples long")
    print("silence covers:",round(percentSilence,4),"%")
    print("notes span range:",noteRange)
    print("max note value:",maxVal)
    print("min note value:",minVal)
    print("average number of notes per sample:",round(avgNotes,4))
    print("average number of notes per sample (adjusted to remove silence samples):",round(adjAvg,4))
    print("max number of notes played in a sample:",maxNotes)
    print("\n")

#NOTE some minor discrepencies vs reading in from generated file directly
#However this does provide a uniform check to use for songs generated by both encoding schemes
#Can also be used to evaluate training file
#uses split encoding to get the text representation for ease of development

In [ ]:
#@title Basic graph of the last output
seconds_to_show = 30 #@param {type:"slider", min:1, max:180, step:1}
show_whole_track = False #@param {type:"boolean"}
graph_color = "red" #@param ["blue", "red", "green"]

x = []
y = []
z = []

t = 0
itrack = 1
fig = plt.figure(figsize=(12,5))
while itrack < len(output):
  for event in output[itrack]:
      if event[0] == 'note':
        y.append(event[4])
        x.append(t)
        plt.plot(x, y, color=graph_color)
        t += 0.25       
        if not show_whole_track:
          if t == seconds_to_show: break
  itrack +=1
plt.show()

In [ ]:
#@title Output MIDI bokeh plot

preset = Preset(plot_width=850)
plotter = Plotter(preset, plot_max_length_bar=4)

pm = PrettyMIDI("/content/Meddleying-VIRTUOSO-Composition.mid")
plotter.show_notebook(pm)

In [ ]:
#@title Bonus Music21 Graphs (slow)
display_relative_output = True
histogram_pitchSpace_count = False #@param {type:"boolean"}
histogram_pitchClass_count = False #@param {type:"boolean"}
Windowed_Keys = False #@param {type:"boolean"}
scatter_quarterLength_pitchSpace = True #@param {type:"boolean"}
quarterLength_3DBars_pitchSpace_count = False #@param {type:"boolean"}

s = converter.parse("/content/Meddleying-VIRTUOSO-Composition.mid")
p = 0
if histogram_pitchSpace_count:
  p = music21.graph.plot.HistogramPitchSpace(s)
  p.id
  'histogram-pitchSpace-count'
  p.run()  # with defaults and proper configuration, will open graph

if histogram_pitchClass_count:
  p = graph.plot.HistogramPitchClass(s)
  p.id
  'histogram-pitchClass-count'
  p.run()

if Windowed_Keys:
  p = graph.plot.WindowedKey(s.parts[0])
  p.id
  p.run()

if scatter_quarterLength_pitchSpace:
  p = graph.plot.ScatterPitchSpaceQuarterLength(s)
  p.id
  'scatter-quarterLength-pitchSpace'
  p.run()
if quarterLength_3DBars_pitchSpace_count:
  p = graph.plot.Plot3DBarsPitchSpaceQuarterLength(s)
  p.id
  '3DBars-quarterLength-pitchSpace-count'
  p.run()
if p == 0:
  print('Please select graph(s) to plot, please :)')

# Congrats! :) You did it :)

In [ ]:
#@title Make a nice Arc diagram of the output to show friends and family :)
MIDI_file_track_to_visualize = 0
multi_track_input = True

midi_file = '/content/Meddleying-VIRTUOSO-Composition.mid'
plot_title = "Meddleying VIRTUOSO Output Arc Diagram"

def maximal_matching_pair( s, substring_length, old_index=-1 ):
    '''
    find the first pair of matching substrings at least as long as the specified length
    '''
    if substring_length > len(s)/2:
        return (len(substring_length), -1) # fail- futile to keep searching with this string

    head = s[:substring_length]
    tail = s[substring_length:]
    index = tail.find(head) 
    if index == -1:
        if substring_length > 2:
            return (substring_length-1, old_index) # success
        return (substring_length, index) # fail- failed on first 2 character substring attempt 
    
    return maximal_matching_pair(s, substring_length+1, index) # keep looking

def first_matching_substring_pair( s, start=0 ):
    '''
    returns the first matching substring pair of at least length 2 in the given string, 
    ignoring all characters of the string before the given start index 
    '''
    if start < 0:
        return () # invalid input: start must be non-negative

    if len(s[start:]) < 4:
        return () # fail: string too short to find matching substrings of minimal length 2

    minimal_substring_length = 2
    (length, distance) = maximal_matching_pair(s[start:], minimal_substring_length)
    if distance != -1:
        return (start, length, distance) # success
    
    return first_matching_substring_pair(s, start+1) # keep looking

def matching_substring_pairs( string ):
    '''
    returns a collection of consecutive substring pairs encoded as (start, length, distance) where
    * start is the index of the first character of the first substring of the matching substring pair,
    * length is the length of the substrings in the matching substring pair, and
    * distance is the distance from the end of the first substring to the begining of the second substring
    '''
    pairs = []
    pair = first_matching_substring_pair(string, 0)
    while pair:
        pairs.append(pair)
        (start, length, distance) = pair
        pair = first_matching_substring_pair(string, start+length)
    return pairs

def plot_arc_diagram( string, plot_title="" ):
    slds = matching_substring_pairs(string)
    bews = map( lambda sld: (sld[0], sum(sld)+sld[1], sld[1]), slds )
    plot_arc_diagram_impl(bews, plot_title)

#  begin                                        end                 
# /                                            /
# ***********-----------O-----------***********
# |--width--|            \          |--width--|
#           |-inner rad-| \
# |-----outer radius----|  center

def plot_ring( ax, begin, end, width ):
    cx = 0.5*(begin + end)
    center = (cx, 0)
    outer_radius = cx - begin
    inner_radius = outer_radius - width

    mypie, _ = ax.pie([1], radius=outer_radius, colors=[(0.4,0.4, 1.0, 0.3)], center=center )
    plt.setp( mypie, width=width)

    return outer_radius

def plot_arc_diagram_impl( bews, plot_title ):
    fig, ax = plt.subplots(subplot_kw={'aspect': 'auto'})

    x_min = 0
    x_max = 1920
    max_width = 1080
    for bew in bews:
        x_max = max(x_max, bew[1])
        orad = plot_ring(ax, bew[0], bew[1], bew[2])
        max_width = max(max_width, orad)

    ax.set_xlim(x_min, x_max)
    ax.set_ylim( -max_width, max_width)

    plt.axis('off')

    title_obj = plt.title(plot_title, loc='center')
    plt.setp(title_obj, color=(0.0, 0.0, 0.0, 1)) 

    plt.savefig('/content/Meddleying-VIRTUOSO-Arc-Diagram.png', dpi=600)
    plt.show()


def stringify_notes(midi_file, track_number):

    mid = MidiFile(midi_file)
    track_notes = {}
    for i, track in enumerate(mid.tracks):
        track_notes[i] = ''
        for msg in track:
            if( msg.type == 'note_on'):
                track_notes[i] += str(msg.note) +'n'
            if( msg.type == 'note_off'):
                track_notes[i] += str(msg.note) +'f'
    return track_notes[track_number]

if multi_track_input:
  try:
    plot_arc_diagram(stringify_notes(midi_file, MIDI_file_track_to_visualize), plot_title)
    if debug: 
      print('Debug mode')
    print('MIDI Track #', MIDI_file_track_to_visualize, 'Arc Diagram')
    Image('output-absolute.png')
  except:
    print('Error in processing your MIDI file. Sorry.')
    sys.exit
from google.colab import files
files.download('/content/Meddleying-VIRTUOSO-Arc-Diagram.png')